# Calculate aggregate engagement metrics and get top posts

This notebook loads the raw data produced by Facebook's API, and then calculates the following: 

- Each page's number of posts and total engagement for each week and month between `2015-01-01` and `2017-03-31`, and for several specific time periods


- The total engagement for the 100 most popular posts on the left and right each week


- The top 50 posts overall, and by post type (photo, video, etc.)


In [1]:
import pandas as pd
from datetime import datetime, timedelta
from glob import glob

## Load the raw data

In [2]:
status_paths = glob("../data/statuses/*.csv")
len(status_paths)

452

In [3]:
def parse_status_csv(path):
    return pd.read_csv(
        path,
        parse_dates=['status_published'],
        date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'),
    ).assign(page_id=lambda x: x["status_id"].str.split("_").str[0])

In [4]:
_statuses = pd.concat([ parse_status_csv(path) for path in status_paths ])\
    .drop_duplicates(subset=["status_id"])

In [5]:
len(_statuses)

4022667

## Select only statuses posted between `2015-01-01` and `2017-03-31`, and remove cover photos

(Changes to a page's cover photo are counted as posts in Facebook's API data.)

In [6]:
cover_photos = _statuses[
    (_statuses["status_type"] == "photo") &
    _statuses["status_link"].str.contains(r"^https://www.facebook.com") &
    _statuses["link_name"].str.contains(r" cover photo$")
]
len(cover_photos)

1917

In [7]:
cover_photos["link_name"].value_counts().head()

The Republican Coffee Corner with Angela's cover photo    411
Truthdig's cover photo                                    277
Real Progressives's cover photo                           100
Being Liberal's cover photo                                45
WND's cover photo                                          38
Name: link_name, dtype: int64

In [8]:
to_date = lambda x: datetime.strptime(x, "%Y-%m-%d").date()

In [9]:
statuses = _statuses[
    ~_statuses["status_id"].isin(set(cover_photos["status_id"])) &
    (_statuses["status_published"].dt.date >= to_date("2015-01-01")) &
    (_statuses["status_published"].dt.date < to_date("2017-04-01"))
]

len(statuses)

4020255

## Add partisanship categorizations

In [10]:
page_info = pd.read_csv(
    '../output/fb-page-info-summary.csv',
    dtype={ "page_id": str },
)

In [11]:
statuses = pd.merge(
    page_info[[ "page_id", "page_name", "political_category" ]],
    statuses,
    on="page_id",
    how="left"
)

In [12]:
# Should be same number as before.
len(statuses)

4020255

### Pages with missing `political_category`

(There should be none.)

In [13]:
print("\n".join(statuses[
    statuses["political_category"].isnull()
]["page_id"].unique()))

## Calculate  `total_engagement` for each post

In [14]:
METRIC_COLUMNS = [ c for c in statuses.columns if "num_" in c ]

In [15]:
statuses[METRIC_COLUMNS] = statuses[METRIC_COLUMNS].astype(int)

In [16]:
statuses["total_engagement"] = statuses[[ "num_reactions", "num_comments", "num_shares" ]].sum(axis=1)

## Calculate metrics by week and month

In [17]:
def aggregate(statuses, groups, metric_fn=sum):
    grp = statuses.groupby(groups)
    df = grp[["total_engagement"] + METRIC_COLUMNS ].agg(metric_fn)
    df["post_count"] = grp.size()
    return df

In [18]:
def aggregate_pages_by_time(statuses, frequency, metric_fn=sum):
    time_grouper = pd.Grouper(
        key="status_published",
        freq=frequency,
        closed="left",
        label="left"
    )
    return aggregate(statuses, [ "page_id", time_grouper ], metric_fn)

### Weekly

In [19]:
weekly_totals = aggregate_pages_by_time(statuses, "W-SUN").astype(int)
weekly_totals.head()

total_engagement  num_reactions  \
page_id         status_published                                    
100434040001314 2014-12-28                    2507           1733   
                2015-01-04                    8534           6149   
                2015-01-11                    9975           6978   
                2015-01-18                   11411           8491   
                2015-01-25                    8772           6242   

                                  num_comments  num_shares  num_likes  \
page_id         status_published                                        
100434040001314 2014-12-28                 255         519       1733   
                2015-01-04                 788        1597       6149   
                2015-01-11                 839        2158       6978   
                2015-01-18                 920        2000       8491   
                2015-01-25                 863        1667       6242   

                                  num_loves  num_wows  num_hahas  num_sads  \
page_id         status_published                                             
100434040001314 2014-12-28                0         0          0         0   
                2015-01-04                0         0          0         0   
                2015-01-11                0         0          0         0   
                2015-01-18                0         0          0         0   
                2015-01-25                0         0          0         0   

                                  num_angrys  post_count  
page_id         status_published                          
100434040001314 2014-12-28                 0          20  
                2015-01-04                 0          74  
                2015-01-11                 0          78  
                2015-01-18                 0          77  
                2015-01-25                 0          72

In [20]:
weekly_totals.to_csv("../output/page-engagement-by-week.csv")

#### Just for Fox News and Occupy Democrats

In [21]:
fox_and_occupy_ids = page_info[
    page_info["page_name"].isin([ "Fox News", "Occupy Democrats" ])
].set_index("page_id")["page_name"].to_dict()
fox_and_occupy_ids

{'15704546335': 'Fox News', '346937065399354': 'Occupy Democrats'}

In [22]:
fox_vs_occupy_weekly_engagement = weekly_totals.loc[list(fox_and_occupy_ids.keys())]\
    ["total_engagement"].unstack(level=0)\
    .rename(columns=fox_and_occupy_ids).astype(int)

fox_vs_occupy_weekly_engagement.head()

page_id,Fox News,Occupy Democrats
status_published,,
2014-12-28,1166366,962809
2015-01-04,3486540,357919
2015-01-11,4479245,875899
2015-01-18,3888642,1042961
2015-01-25,2287591,488994


In [23]:
fox_vs_occupy_weekly_engagement.to_csv("../output/fox-vs-occupy-engagement-by-week.csv")

### Monthly

In [24]:
monthly_totals = aggregate_pages_by_time(statuses, "MS").astype(int)
monthly_totals.head()

total_engagement  num_reactions  \
page_id         status_published                                    
100434040001314 2015-01-01                   41199          29593   
                2015-02-01                   42369          30672   
                2015-03-01                   57146          41921   
                2015-04-01                   67353          50715   
                2015-05-01                   69072          49787   

                                  num_comments  num_shares  num_likes  \
page_id         status_published                                        
100434040001314 2015-01-01                3665        7941      29593   
                2015-02-01                3873        7824      30672   
                2015-03-01                4773       10452      41921   
                2015-04-01                5116       11522      50715   
                2015-05-01                5682       13603      49787   

                                  num_loves  num_wows  num_hahas  num_sads  \
page_id         status_published                                             
100434040001314 2015-01-01                0         0          0         0   
                2015-02-01                0         0          0         0   
                2015-03-01                0         0          0         0   
                2015-04-01                0         0          0         0   
                2015-05-01                0         0          0         0   

                                  num_angrys  post_count  
page_id         status_published                          
100434040001314 2015-01-01                 0         321  
                2015-02-01                 0         346  
                2015-03-01                 0         388  
                2015-04-01                 0         455  
                2015-05-01                 0         441

In [25]:
monthly_totals.to_csv("../output/page-engagement-by-month.csv")

## Get top 100 posts by week and partisanship

In [26]:
def get_top_100_posts_total_engagement(df):
    top = df.nlargest(100, "total_engagement")
    page_counts = top["page_id"].value_counts()
    total_engagement = top["total_engagement"].sum()
    return pd.Series({
        "num_posts": len(top),
        "total_engagement": total_engagement,
        "max_engagement": top["total_engagement"].max(),
        "min_engagement": top["total_engagement"].min(),
        "distinct_pages": len(page_counts),
        "top_page_count": page_counts.iloc[0],
        "top_page_prop_engagement": top.groupby("page_id")["total_engagement"].sum().max()\
            / total_engagement,
    })

In [27]:
weekly_top_posts_by_partisanship = statuses.groupby([
    pd.Grouper(
        key="status_published",
        freq="W-SUN",
        closed="left",
        label="left"
    ),
    "political_category",
]).apply(get_top_100_posts_total_engagement).unstack()

weekly_top_posts_by_partisanship.head()

distinct_pages       max_engagement             \
political_category           left right           left      right   
status_published                                                    
2014-12-28                   21.0  22.0       655862.0  1608308.0   
2015-01-04                   23.0  21.0       219835.0   598736.0   
2015-01-11                   20.0  19.0       359320.0  1828325.0   
2015-01-18                   21.0  21.0       581405.0  1744063.0   
2015-01-25                   17.0  20.0       172044.0  1749288.0   

                   min_engagement          num_posts        top_page_count  \
political_category           left    right      left  right           left   
status_published                                                             
2014-12-28                 8365.0  21088.0     100.0  100.0           16.0   
2015-01-04                15625.0  42047.0     100.0  100.0           15.0   
2015-01-11                15091.0  48988.0     100.0  100.0           21.0   
2015-01-18                17212.0  46629.0     100.0  100.0           15.0   
2015-01-25                14531.0  38044.0     100.0  100.0           19.0   

                         top_page_prop_engagement           total_engagement  \
political_category right                     left     right             left   
status_published                                                               
2014-12-28          22.0                 0.337049  0.249237        2735721.0   
2015-01-04          24.0                 0.221724  0.274383        3073663.0   
2015-01-11          27.0                 0.221967  0.293881        4198339.0   
2015-01-18          29.0                 0.201163  0.441159        4220414.0   
2015-01-25          25.0                 0.280049  0.225935        2950189.0   

                                
political_category       right  
status_published                
2014-12-28           6669197.0  
2015-01-04          10325893.0  
2015-01-11          14309438.0  
2015-01-18          14749326.0  
2015-01-25           9383113.0

In [28]:
weekly_top_posts_by_partisanship[[
    "distinct_pages",
    "top_page_count",
    "top_page_prop_engagement",
    "min_engagement"
]].describe()

distinct_pages             top_page_count              \
political_category           left       right           left       right   
count                  118.000000  118.000000     118.000000  118.000000   
mean                    14.983051   23.686441      40.279661   26.906780   
std                      3.343531    4.962594      11.742163    6.292021   
min                      9.000000   12.000000      15.000000   16.000000   
25%                     12.000000   20.000000      33.250000   22.000000   
50%                     15.000000   24.000000      41.500000   26.000000   
75%                     17.000000   27.000000      47.750000   30.750000   
max                     23.000000   34.000000      64.000000   46.000000   

                   top_page_prop_engagement             min_engagement  \
political_category                     left       right           left   
count                            118.000000  118.000000     118.000000   
mean                               0.398698    0.284040   38809.177966   
std                                0.103203    0.077730   19829.731583   
min                                0.201163    0.156508    8365.000000   
25%                                0.319790    0.225949   23556.250000   
50%                                0.400200    0.271161   38478.000000   
75%                                0.468442    0.336963   48094.500000   
max                                0.687477    0.481636  101720.000000   

                                   
political_category          right  
count                  118.000000  
mean                 57408.440678  
std                  16763.954391  
min                  21088.000000  
25%                  46300.500000  
50%                  53257.500000  
75%                  63175.500000  
max                 145990.000000

In [29]:
# Make sure that we have at least 100 posts per week for both left/right
assert (weekly_top_posts_by_partisanship["num_posts"] < 100).sum().sum() == 0

In [30]:
weekly_top_posts_by_partisanship["total_engagement"]\
    .astype(int)\
    .to_csv("../output/top-100-posts-total-engagement-by-week-and-partisanship.csv")

## Get top 50 posts, overall, by post type


In [31]:
def get_top(df, num):
    return df.sort_values([
        "total_engagement",
        "status_id"
    ], ascending=False).head(num)

In [32]:
for status_type, group in statuses.groupby([ "status_type" ]):
    print(status_type)
    get_top(group, 50).to_csv(
        "../output/top-50-posts-{}.csv".format(status_type),
        index=False
    )

event
link
music
note
offer
photo
status
video


### ... and among all posts

In [33]:
get_top(statuses, 50).to_csv("../output/top-50-posts-all.csv", index=False)

---

---

---